In [2]:
import os
os.listdir()
os.getcwd()


'f:\\Studying_Materials\\Dai_Hoc_Quoc_Te\\Studying\\Year_04_2526\\Semester_01_2526\\Data_Mining\\Projects\\Flims\\pre-processing'

In [3]:
import pandas as pd
import numpy as np

anime = pd.read_csv("../dataset/raw/anime.csv")
rating = pd.read_csv("../dataset/raw/rating.csv")

anime.head(), rating.head()


(   anime_id                              name  \
 0     32281                    Kimi no Na wa.   
 1      5114  Fullmetal Alchemist: Brotherhood   
 2     28977                          Gintama°   
 3      9253                       Steins;Gate   
 4      9969                     Gintama&#039;   
 
                                                genre   type episodes  rating  \
 0               Drama, Romance, School, Supernatural  Movie        1    9.37   
 1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
 2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
 3                                   Sci-Fi, Thriller     TV       24    9.17   
 4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   
 
    members  
 0   200630  
 1   793665  
 2   114262  
 3   673572  
 4   151266  ,
    user_id  anime_id  rating
 0        1        20      -1
 1        1        24      -1
 2        1        79      -1
 3   

In [4]:
# episodes: thay 'Unknown' bằng NaN và chuyển sang số
anime['episodes'] = anime['episodes'].replace('Unknown', np.nan).astype(float)

# genre: tách theo dấu phẩy thành list
anime['genre'] = anime['genre'].fillna("").apply(lambda x: x.split(", "))

# rating: convert sang float
anime['rating'] = pd.to_numeric(anime['rating'], errors='coerce')

# fill NaN rating bằng mean hoặc median
anime['rating'] = anime['rating'].fillna(anime['rating'].mean())


In [5]:
anime.info()
anime.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  11954 non-null  float64
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 672.5+ KB


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1.0,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64.0,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24.0,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.16,151266


In [7]:
rating['rating'] = rating['rating'].replace(-1, np.nan)
rating = rating.dropna()
rating.head()


,user_id,anime_id,rating
47,1,8074,10.0
81,1,11617,10.0
83,1,11757,10.0
101,1,15451,10.0
153,2,11771,10.0


In [8]:
rating['rating'].describe()


count    6.337241e+06
mean     7.808497e+00
std      1.572496e+00
min      1.000000e+00
25%      7.000000e+00
50%      8.000000e+00
75%      9.000000e+00
max      1.000000e+01
Name: rating, dtype: float64

In [10]:
merged = rating.merge(anime, on="anime_id", how="left")
merged.head()


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,8074,10.0,Highschool of the Dead,"[Action, Ecchi, Horror, Supernatural]",TV,12.0,7.46,535892.0
1,1,11617,10.0,High School DxD,"[Comedy, Demons, Ecchi, Harem, Romance, School]",TV,12.0,7.70,398660.0
2,1,11757,10.0,Sword Art Online,"[Action, Adventure, Fantasy, Game, Romance]",TV,25.0,7.83,893100.0
3,1,15451,10.0,High School DxD New,"[Action, Comedy, Demons, Ecchi, Harem, Romance...",TV,12.0,7.87,266657.0
4,2,11771,10.0,Kuroko no Basket,"[Comedy, School, Shounen, Sports]",TV,25.0,8.46,338315.0


In [11]:
anime.to_csv("../dataset/processed/anime_cleaned.csv", index=False)
rating.to_csv("../dataset/processed/rating_cleaned.csv", index=False)
merged.to_csv("../dataset/processed/merged.csv", index=False)


In [12]:
!pip install liac-arff


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11782 sha256=e05e787cff00ebe199b498691cf564c9af7bcad68964da56050d44669b8ff7d6
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\93\f3\5b\658a9bddee916a5f4b84bdc1a4e0fabd22fb17947c2c9542e6
Successfully built liac-arff



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
merged['genre'] = merged['genre'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
merged = merged.replace({np.nan: "?"})
def escape(s):
    if isinstance(s, str):
        s = s.replace("'", "\\'")
        return f"'{s}'"
    return s

for col in merged.columns:
    if merged[col].dtype == object:
        merged[col] = merged[col].apply(escape)


In [17]:
import arff

def to_arff(df, filename):
    # attributes
    attributes = []
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            attributes.append((col, 'NUMERIC'))
        else:
            attributes.append((col, 'STRING'))
    
    data = {
        'description': '',
        'relation': 'anime_dataset',
        'attributes': attributes,
        'data': df.values.tolist()
    }

    with open(filename, 'w', encoding='utf-8') as f:
        arff.dump(data, f)

to_arff(merged, "../dataset/processed/merged.arff")
